## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [14]:
import tensorflow as tf
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 权重
        self.W1 = tf.Variable(tf.random.normal(shape=[28*28, 100]), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=[100, 10]), trainable=True)

        # 偏置
        self.b1 = tf.Variable(tf.zeros(shape=[100]), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=[10]), trainable=True)
    def forward(self, x):
        x = tf.reshape(x, [-1, 28*28])
        
        h1 = tf.matmul(x, self.W1) + self.b1
        h1_relu = tf.nn.relu(h1)
        h2 = tf.matmul(h1_relu, self.W2) + self.b2
        h2_log = tf.nn.log_softmax(h2)
        return h2_log
    def backward(self):
        pass
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################

        return self.forward(x)
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [12]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [20]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 4.873097 ; accuracy 0.7700833
epoch 1 : loss 4.869285 ; accuracy 0.77015
epoch 2 : loss 4.865483 ; accuracy 0.77025
epoch 3 : loss 4.8616905 ; accuracy 0.77031666
epoch 4 : loss 4.8579073 ; accuracy 0.7704
epoch 5 : loss 4.854132 ; accuracy 0.7706
epoch 6 : loss 4.850366 ; accuracy 0.7707
epoch 7 : loss 4.846608 ; accuracy 0.7707667
epoch 8 : loss 4.8428593 ; accuracy 0.7708833
epoch 9 : loss 4.839119 ; accuracy 0.77103335
epoch 10 : loss 4.8353853 ; accuracy 0.77105
epoch 11 : loss 4.8316603 ; accuracy 0.77105
epoch 12 : loss 4.8279443 ; accuracy 0.77115
epoch 13 : loss 4.8242364 ; accuracy 0.77135
epoch 14 : loss 4.820537 ; accuracy 0.7715
epoch 15 : loss 4.816846 ; accuracy 0.77165
epoch 16 : loss 4.8131623 ; accuracy 0.77185
epoch 17 : loss 4.8094873 ; accuracy 0.77195
epoch 18 : loss 4.80582 ; accuracy 0.77201664
epoch 19 : loss 4.8021603 ; accuracy 0.77208334
epoch 20 : loss 4.7985106 ; accuracy 0.77225
epoch 21 : loss 4.794868 ; accuracy 0.77236664
epoch 22 : loss